# Type of examples for which stop words predict the wrong relation vs the baseline which predicts the correct relations

In [3]:
%load_ext autoreload

In [16]:
%autoreload
import os
from sys import path
import re
import pandas as pd
path.append('../../..')
import numpy as np
from scipy.stats import ttest_rel
from relation_extraction.data.converters.converter_i2b2 import relation_dict
output_path = '/scratch/geeticka/relation-extraction-result/i2b2-analyze/'
def res(path): return os.path.join(output_path, path)
original_sentences_path = os.path.join('/crimea/geeticka/data/relation_extraction/i2b2/pre-processed/original/test_original.txt')
stop_words_sentences_path = os.path.join('/crimea/geeticka/data/relation_extraction/i2b2/pre-processed/punct_stop_digit/test_punct_stop_digit.txt')

In [17]:
relation_dict

{0: 'TrIP',
 1: 'TrWP',
 2: 'TrCP',
 3: 'TrAP',
 4: 'TrNAP',
 5: 'TeRP',
 6: 'TeCP',
 7: 'PIP',
 8: 'None'}

In [18]:
def read_answers_line(line):
    linenum, relation = line.strip().split()
    return linenum, relation

In [19]:
def asstring(list_of_strings):
    return " ".join(list_of_strings)

In [20]:
def read_sentence_and_entities(line):
    line = line.strip().split()
    sentence = line[5:]
    relation = relation_dict[int(line[0])]
    entity1_idx = (int(line[1]), int(line[2]))
    entity2_idx = (int(line[3]), int(line[4]))
    entity1 = sentence[entity1_idx[0] : entity1_idx[1] + 1]
    entity2 = sentence[entity2_idx[0] : entity2_idx[1] + 1]
    
    return relation, asstring(entity1), asstring(entity2), asstring(sentence)

In [21]:
needed_linenum_and_relation = {}

In [22]:
with open(res('answers_for_dev-stop-words.txt')) as textfile1, open(res("answers_for_dev-baseline.txt")) as textfile2, \
open(res('answers_for_dev_gold-baseline.txt')) as textfile3: 
    for x, y, z in zip(textfile1, textfile2, textfile3):
        linenum, stop_words_relation = read_answers_line(x)
        _, baseline_relation = read_answers_line(y)
        _, gold_relation = read_answers_line(z)
        if baseline_relation == gold_relation and stop_words_relation != gold_relation:
            needed_linenum_and_relation[int(linenum) - 1] = (baseline_relation, stop_words_relation)

In [23]:
len(list(needed_linenum_and_relation.keys()))

1399

Note that we list baseline first and then the stop words version but the prediction of the baseline version is correct (gold) whereas the stop words is incorrect. 

In [24]:
print('We print the baseline first and then the stop words version. Gold relation corresponds to baseline\n\n')
curr_linenum = 0
with open(original_sentences_path) as original_sentences, open(stop_words_sentences_path) as stop_words_sentences:
    for x, y in zip(original_sentences, stop_words_sentences):
        needed_linenums = list(needed_linenum_and_relation.keys())
        if curr_linenum in needed_linenums:
            _, e1_b, e2_b, s_b = read_sentence_and_entities(x.strip())
            _, e1_c, e2_c, s_c = read_sentence_and_entities(y.strip())
            r_b, r_c = needed_linenum_and_relation[curr_linenum]
            print('Predicted Relation: \t {0}, {1} \nEntities: \t {2}, {3} \t {4}, {5} \nSentences: \n\t{6} \n\t {7}'.format(
                    r_b, r_c, e1_b, e1_c, e2_b, e2_c, s_b, s_c))
            print('\n')
        curr_linenum += 1

We print the baseline first and then the stop words version. Gold relation corresponds to baseline


Predicted Relation: 	 None, TrAP 
Entities: 	 coronary artery bypass graft, coronary artery bypass graft 	 anterior myocardial infarction, anterior myocardial infarction 
Sentences: 
	2 ) Coronary artery disease , status post coronary artery bypass graft , status post anterior myocardial infarction , sick sinus syndrome , status post a VVI pacer placement . 
	 NUMBER Coronary artery disease status post coronary artery bypass graft status post anterior myocardial infarction sick sinus syndrome status post VVI pacer placement


Predicted Relation: 	 TrAP, None 
Entities: 	 vitamin C, vitamin C 	 postoperative anemia, postoperative anemia 
Sentences: 
	Iron and vitamin C were started for postoperative anemia . 
	 Iron vitamin C started postoperative anemia


Predicted Relation: 	 TeRP, TrAP 
Entities: 	 injection of agitated saline contrast, injection agitated saline contrast 	 A right-to-

Predicted Relation: 	 None, TrAP 
Entities: 	 Colace, Colace 	 pain, pain 
Sentences: 
	Tylenol 650 mg p.o. q.4h. p.r.n. , Benadryl 25 mg p.o. q.h.s. p.r.n. , Colace 100 mg p.o. q.i.d. , Nortriptyline 25 mg p.o. q.h.s. , Simvastatin 10 mg p.o. q.h.s. , Metamucil one packet p.o. b.i.d. p.r.n. , Neurontin 300 mg p.o. t.i.d. , Levsinex 0.375 mg p.o. q.12h. , Lisinopril / hydrochlorothiazide 20/25 mg p.o. q.d. , hydrocortisone topical ointment to affected areas , MS Contin 30 mg p.o. b.i.d. , MSIR 15 to 30 mg p.o. q.4h. p.r.n. pain . 
	 Tylenol NUMBER mg p.o. q.4h. p.r.n. Benadryl NUMBER mg p.o. q.h.s. p.r.n. Colace NUMBER mg p.o. q.i.d. Nortriptyline NUMBER mg p.o. q.h.s. Simvastatin NUMBER mg p.o. q.h.s. Metamucil packet p.o. b.i.d. p.r.n. Neurontin NUMBER mg p.o. t.i.d. Levsinex NUMBER mg p.o. q.12h. Lisinopril hydrochlorothiazide NUMBER mg p.o. q.d. hydrocortisone topical ointment affected areas MS Contin NUMBER mg p.o. b.i.d. MSIR NUMBER NUMBER mg p.o. q.4h. p.r.n. pain


Predicted Re



Predicted Relation: 	 PIP, TrIP 
Entities: 	 Duke &apos;s grade B-II, Duke &apos;s grade B-II 	 colon carcinoma, colon carcinoma 
Sentences: 
	In December of 1992 , she underwent a transverse colectomy for colon carcinoma which was Duke &apos;s grade B-II , lymph nodes were negative . 
	 December NUMBER underwent transverse colectomy colon carcinoma Duke &apos;s grade B-II lymph nodes negative


Predicted Relation: 	 TeCP, TeRP 
Entities: 	 bronchoscopy, bronchoscopy 	 a mass in the left superhilar region, mass left superhilar region 
Sentences: 
	The patient is a 66 year old white female with metastatic colon carcinoma with a mass in the left superhilar region , who presents for bronchoscopy , mediastinal exploration and resection . 
	 patient NUMBER year old white female metastatic colon carcinoma mass left superhilar region presents bronchoscopy mediastinal exploration resection


Predicted Relation: 	 TeRP, None 
Entities: 	 her blood, blood 	 an increased level of CEA, increased

Predicted Relation: 	 None, TrAP 
Entities: 	 Tylenol, Tylenol 	 perianal irritation, perianal irritation 
Sentences: 
	Tylenol 650 mg p.o. q. 4-6h p.r.n. headache or pain ; acyclovir 400 mg p.o. t.i.d.; acyclovir topical t.i.d. to be applied to lesion on corner of mouth ; Peridex 15 ml p.o. b.i.d.; Mycelex 1 troche p.o. t.i.d.; g-csf 404 mcg subcu q.d.; folic acid 1 mg p.o. q.d.; lorazepam 1-2 mg p.o. q. 4-6h p.r.n. nausea and vomiting ; Miracle Cream topical q.d. p.r.n. perianal irritation ; Eucerin Cream topical b.i.d.; Zantac 150 mg p.o. b.i.d.; Restoril 15-30 mg p.o. q. h.s. p.r.n. insomnia ; multivitamin 1 tablet p.o. q.d.; viscous lidocaine 15 ml p.o. q. 3h can be applied to corner of mouth or lips p.r.n. pain control . 
	 Tylenol NUMBER mg p.o. q. 4-6h p.r.n. headache pain acyclovir NUMBER mg p.o. t.i.d.; acyclovir topical t.i.d. applied lesion corner mouth Peridex NUMBER ml p.o. b.i.d.; Mycelex NUMBER troche p.o. t.i.d.; g-csf NUMBER mcg subcu q.d.; folic acid NUMBER mg p.o. q

Predicted Relation: 	 None, PIP 
Entities: 	 a URI, URI 	 fever, fever 
Sentences: 
	Pt was o/w in his USOH until 2-3 weeks ago when he developed a URI with symptoms of dry cough; no fever , chills , sore throat , myalgias . 
	 Pt o/w USOH 2-3 weeks ago developed URI symptoms dry cough; fever chills sore throat myalgias


Predicted Relation: 	 None, PIP 
Entities: 	 symptoms, symptoms 	 myalgias, myalgias 
Sentences: 
	Pt was o/w in his USOH until 2-3 weeks ago when he developed a URI with symptoms of dry cough; no fever , chills , sore throat , myalgias . 
	 Pt o/w USOH 2-3 weeks ago developed URI symptoms dry cough; fever chills sore throat myalgias


Predicted Relation: 	 None, PIP 
Entities: 	 a URI, URI 	 myalgias, myalgias 
Sentences: 
	Pt was o/w in his USOH until 2-3 weeks ago when he developed a URI with symptoms of dry cough; no fever , chills , sore throat , myalgias . 
	 Pt o/w USOH 2-3 weeks ago developed URI symptoms dry cough; fever chills sore throat myalgias


Predicte